| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

This notebook is licensed under GPL 3.0. Please visit our [Github repo](https://github.com/edgi-govdata-archiving/ECHO-Cross-Program) for more information.

The notebook was collaboratively authored by EDGI following our [authorship protocol](https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/).

For more information about this project, visit https://www.environmentalenforcementwatch.org/

Note:  This notebook pulls data from a copy of EPA's ECHO database hosted by Stony Brook University. The data sets are updated on a weekly basis, meaning that some of the results from your run may not exactly match those in [EEW's Congressional Report Cards](https://www.environmentalenforcementwatch.org/reports). For instance, the Report Cards show ten facilities that have spent at least three of the past 12 quarters in non-compliance with different environmental protection laws. These results will therefore change as we enter new parts of the year. In addition, the Report Cards estimate the number of facilities that were active in 2019, since EPA does not provide such figures. Our estimate is based on the number of facilities EPA records as active at the *current* moment in time. In short, we use active right now as a proxy for active in 2019. This number informs several metrics in the Report Cards - including violations and inspections per 1000 facilities - and these will change as the number of facilities reported as active right now by the EPA changes. Please see the [CD-Report repo](https://github.com/edgi-govdata-archiving/CD-report) for facility counts and non-compliance rates as we recorded them in mid-September 2020 in order to produce the Report Cards.

# Examining Data from the EPA's Risk Screening Environmental Indicators (RSEI) 

This notebook examines data from the Risk Screening Environmental Indicators (RSEI) database (https://epa.gov/rsei). 

As data is retrieved from each RSEI data set, a subset of the available fields are selected. Those are in the ***column*** variable in the code blocks. 

Additional columns can be added by modifying the list in the ***column*** variable.

The fields available and their meaning can be found in the data dictionary at this link: (https://www.epa.gov/rsei/rsei-data-dictionary-site-data).

In [ ]:
# Install our codebase 
# !pip install ECHO_modules >&/dev/null;
%pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules@neighborhoods >&/dev/null;
%pip install geopandas >&/dev/null;

### Select the type of region and then the state
A state selection is not necessary for Zip Code and Neighborhood region types.

An additional choice lets you supply a list of FRSIDs and use those facilities in the analysis.

In [1]:
from ECHO_modules.get_data import get_echo_data
from ECHO_modules.utilities import show_region_type_widget, \
    show_state_widget, show_year_range_widget
from ECHO_modules.rsei_utilities import show_rsei_pick_region_widget

region_type_widget = show_region_type_widget(region_types=('City', 'County', 'State', 'Zip Code', 'Neighborhood',
                                                           'FRSID List'), 
                                             default_value='City' )
state_widget = None
# display( region_type_widget )
print('(The State will be ignored for Zip Code and Neighborhood regions.)')
state_widget = show_state_widget()

Dropdown(description='Region of interest:', options=('City', 'County', 'State', 'Zip Code', 'Neighborhood', 'F…

(The State will be ignored for Zip Code and Neighborhood regions.)


Dropdown(description='State:', options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI'…

## Select the regions to look for
For Neighborhoods, only rectangles are currently supported.

City, county and state names will be automatically converted to upper case. Don't worry about the case as you type in your selections.

Multiple selections can be made with a comma-separated list.


In [4]:
from ECHO_modules.utilities import polygon_map

description = None
region_widget = None
region_type = region_type_widget.value
if region_type == 'Neighborhood':
    (map,shapes) = polygon_map()
    display(map)
elif region_type != 'State':
    if region_type == 'FRSID List':
        description = 'Select file with FRSID column'
    region_widget = show_rsei_pick_region_widget( type=region_type,
                                            state_widget=state_widget, 
                                            description=description )

Text(value='', description='Select file with FRSID column')

## Get the facilities for the chosen regions
These are the producers of toxic waste in the chosen region, as reported to the EPA's Toxic Release Inventory (TRI).

In [5]:
from ECHO_modules.rsei_utilities import get_rsei_facilities, get_frsid_list, get_this_by_that

state = state_widget.value if state_widget is not None else None
regions_selected = None
if region_type == 'Zip Code':
    regions_selected = str(region_widget.value)
elif region_type == 'Neighborhood':
    regions_selected = shapes.pop()
elif region_type == 'FRSID List':
    regions_selected = get_frsid_list(region_widget.value)
elif region_type != 'State':
    regions_selected = region_widget.value
    
columns = '"FacilityName", "FacilityID", "FacilityNumber", "FRSID", "Latitude", "Longitude", "Street",'
columns += '"City", "County", "State", "ZIPCode", "StandardizedParentCompany"'

if region_type == 'FRSID List':
    fac_df = get_this_by_that(this_name='facility', that_series=regions_selected, this_key='FRSID',
                              this_columns=columns)
else:
    fac_df = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
                                 rsei_type='facility', columns=columns)
# If the columns aren't specified, all columns are returned ("select * from ...")
# fac_df = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
#                              rsei_type='facility')
fac_df

select "FacilityName", "FacilityID", "FacilityNumber", "FRSID", "Latitude", "Longitude", "Street","City", "County", "State", "ZIPCode", "StandardizedParentCompany" from "facility_data_rsei_v2312" where "FRSID" in (110000741243,110017438762,llOOOn35269,110000367567,110056961417,110022447903,110017408296,110003390143,110043787499,110000451038,110000743508,110001148598,110027373045,110041158136,110000611703,110027375775,110031002439,110061066199,110038932411,110011828068,110000432504,110056958430,110000547196,110041962873,110000438884,110000397516,110000404296,110000403670,110015862440,110017424312,110000377976,1100273730n,110021163293,
110009933395,110000380061,110056956511,110008361263,110041978857,110040920242,110000378546,110000378467,110012817354,110000450002,110000450100,110000449970,110003266849,110000597444,110064365690,110064520031,110001316308,
110070827758,110000597319)


HTTPError: HTTP Error 500: Internal Server Error

In [13]:
sql = 'select "FacilityName", "FacilityID", "FacilityNumber", "FRSID", "Latitude", "Longitude", "Street","City", "County", "State", "ZIPCode", "StandardizedParentCompany" from "facility_data_rsei_v2312" where "FRSID" in (110000741243,110017438762,llOOOn35269)'
df = get_echo_data(sql)
df

HTTPError: HTTP Error 500: Internal Server Error

#### See a map of these producing facilities in the regions selected

In [ ]:
from ECHO_modules.utilities import mapper

map_of_facilities = mapper(fac_df, no_text=False, lat_field='Latitude', long_field='Longitude', name_field='FacilityName')
display(map_of_facilities)

#### Choose the years for the submissions you want to see

In [ ]:
from ECHO_modules.utilities import show_year_range_widget
year_range = show_year_range_widget()

We'll work to follow the chain from facilities to their submissions (with the associated chemical),
then from the submission to releases (using SubmissionNumber),
then from releases to elements (using ReleaseNumber),
and from releases to offsite facilities (using releases.OffsiteNumber with offsite.FacilityNumber).
We can then try to connect the offsite facility (offsite.TRIFID) with facility (FacilityID)

### Get the submissions made by these facilities

In [ ]:
from ECHO_modules.rsei_utilities import get_this_by_that

columns = '"SubmissionNumber", "FacilityNumber", "ChemicalNumber", "SubmissionYear", "OneTimeReleaseQty", "TradeSecretInd"'

sub_df = get_this_by_that(this_name='submissions', that_series=fac_df['FacilityNumber'], this_key='FacilityNumber',
                          this_columns=columns, years=year_range.value, year_field='SubmissionYear')
sub_df

Start a linking dataframe with minimal fields to trace from the facility to the offsite locations.
Join on the FacilityNumber fields of fac_df and sub_df (submissions).

In [ ]:
link_df = fac_df.set_index('FacilityNumber').join(sub_df.set_index('FacilityNumber'), lsuffix='_left', rsuffix='_right')
link_df

### Get the releases for the submissions

In [ ]:

columns = '"ReleaseNumber", "SubmissionNumber", "Media", "PoundsReleased", "OffsiteNumber", "TEF"'
rel_df = get_this_by_that(this_name='releases', that_series=sub_df['SubmissionNumber'], this_key='SubmissionNumber',
                          this_columns=columns)
rel_df

Continue the linking process for facilities by joining the previous link with the releases.

In [ ]:
link_df2 = link_df.set_index('SubmissionNumber').join(rel_df.set_index('SubmissionNumber')).dropna(subset=('OffsiteNumber'))
link_df2

### Get the offsite facilities from the releases

In [ ]:

columns = '"FacilityNumber", "TRIFID", "FRSID", "Name", "Street", "City", "State", "ZIPCode", "Latitude", "Longitude"'
off_df = get_this_by_that(this_name='offsite', that_series=rel_df['OffsiteNumber'].dropna(), this_key='OffsiteID', this_columns=columns)
off_df

#### Continue the linking process started earlier. 
This time link the OffsiteNumber from releases with the FacilityNumber in offsite.

In [ ]:
link_df3 = link_df2.set_index('OffsiteNumber').join(off_df.set_index('FacilityNumber'), lsuffix='_left', rsuffix='_right')
link_df3

Pare the linking information down to just the latitude/longitude for the originating facility (_left)
and the coordinates for the offsite facility (_right).
There may be multiple transfers between the same two facilities, so we drop duplicates.
(The multiple transfers may be of interest. They will exist in link_df3.)

In [ ]:
link_df4 = link_df3.drop_duplicates(subset=['Latitude_left', 'Longitude_left', 'Latitude_right', 'Longitude_right'])
link_df4 = link_df4[['Latitude_left', 'Longitude_left', 'Latitude_right', 'Longitude_right']]
link_df4

### Link the producing facilities with their offsite facilities.

Map the facilities releasing and the offsite facilities they send to.
    df_dicts : tuple
        Tuple of dictionaries containing the facilities to map.  They must have a latitude and 
        longitude field. The dictionaries should have these fields:

             the DataFrame - 'DataFrame'

             circle border color - 'marker_color'

             circle interior color - 'marker_fill_color'

             facility name - 'name_field' in the dataframe 

             latitude field - 'lat_field'

             longitude field - 'long_field'

             URL field - 'url_field'

The facilities producing waste will be shown with green circles.
The offsite facilities receiving the waste from the green facilities are shown with blue circles.

Lines show transfer from green dot producing facilities to blue dot offsite facilities.

In [ ]:
from ECHO_modules.rsei_utilities import mapper2

fac_dict = {
    'DataFrame' : fac_df,
    'marker_color' : 'black',
    'marker_fill_color' : 'green',
    'name_field' : 'FacilityName',
    'lat_field' : 'Latitude',
    'long_field' : 'Longitude',
    'url_field' : None
}
off_dict = {
    'DataFrame' : off_df,
    'marker_color' : 'yellow',
    'marker_fill_color' : 'blue',
    'name_field' : 'Name',
    'lat_field' : 'Latitude',
    'long_field' : 'Longitude',
    'url_field' : None
}
map_facs_and_offs = mapper2(df_dicts=(fac_dict, off_dict), link_df=link_df4 )
display(map_facs_and_offs)

## Add the chemicals to the submissions

In [ ]:
from ECHO_modules.rsei_utilities import add_chemical_to_submissions

# columns = '"Chemical", "RfCInhale", "RfDOral"'
columns = '*'
sub1_df = add_chemical_to_submissions(submissions=sub_df, chemical_columns=columns)
columns = ["SubmissionNumber", "ChemicalNumber", "Chemical", "RfCInhale"]
sub1_df[columns]

## Get the elements for the releases

In [ ]:

columns = '"ElementNumber", "PoundsPT", "ScoreCategory", "Score", "Population", "ScoreA", "PopA", "ScoreB", "PopB"'
element_df = get_this_by_that(this_name='elements', that_series=rel_df['ReleaseNumber'], this_key='ReleaseNumber', 
                              this_columns=columns)
element_df

## See offsite facilities for the chosen region
These offsite facilities may be receiving from other facilities outside of this region. They aren't necessarily linked to the producing facilities in fac_df.

In [ ]:
from ECHO_modules.rsei_utilities import get_rsei_facilities

columns = '"Name", "OffsiteID", "FacilityNumber", "TRIFID", "FRSID", "Latitude", "Longitude", "Street",'
columns += '"City", "State", "ZIPCode"'

off_df2 = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
                             rsei_type='offsite', columns=columns)
off_df2

In [ ]:


to_map = off_df2.dropna(subset=['Latitude', 'Longitude'])
map_of_facilities = mapper(to_map, no_text=False, lat_field='Latitude', 
                           long_field='Longitude', name_field='Name')
display(map_of_facilities)

In [ ]:

# All the releases where media = 1 (I think that's direct air releases) 
rsql = 'select * from "releases_data_rsei_v2312" where "Media" <= 2;' 
get_echo_data(rsql)
# All the releases above a certain weight 
rsql = 'select * from "releases_data_rsei_v2312" where "PoundsReleased" > 100000;' 
releases = get_echo_data(rsql)

In [ ]:
len(releases)

In [ ]:
# All the releases where media = 1 (I think that's direct air releases) 
media_sql = 'select "Media", "MediaText" from "media_data_rsei_v2312";' 
media_types = get_echo_data(media_sql)
media_types

In [ ]:
# Get Exxon facilities 
rsql = 'select * from "facility_data_rsei_v2312" where "StandardizedParentCompany" like \'%EXXON%\';' 
facs = get_echo_data(rsql) 
# Get their submissions 
these_fac_numbers = list(facs["FacilityNumber"].unique()) 
rsql = 'select * from "submissions_data_rsei_v2312" where "FacilityNumber" in ({});'.format(','.join([str(fac) for fac in these_fac_numbers])) 
# You shouldn't do SQL like this but I'm being quick 
subs = get_echo_data(rsql) 

# Use these submission numbers to get releases 
# Ok, actually there are too many submissions (>20,000) to easily get all the Exxon releases from the database. 
# An enterprising SQL writer could do this with some joins, I bet! No time right now for me though 
# But this is the general idea.... 
these_submission_numbers = list(subs["SubmissionNumber"].unique())[0:50] 
# Just do the first 50 as a test 
rsql = 'select * from "releases_data_rsei_v2312" where "SubmissionNumber" in ({});'.format(','.join([str(fac) for fac in these_submission_numbers])) 
res = get_echo_data(rsql) 
res